## Personal information

In [1]:
PROJECT_TYPE = 2
NAME = ["Milan Conrad", "Matthias Wilms"]
ID = ["2569325", "2575720"]

In [2]:
IMPLEMENTED = set()

In [3]:
import asyncio
import inspect
import os
import sys
import time
import bookutils
import ipywidgets as widgets

In [4]:
from bookutils import input,next_inputs
import re

## State
This represents the program state at a given point, containing attributes like event-type, line# number etc.
- `rel_lineno` is the line number within a function (i.e def ... is has rel_lineno 1)
- `code` is the code of the current line
- `changed_vars` contains only changed variables
- `fun_code` contains a list of the lines of the function this the program is currently in
- `parent` used for the callstack, references the caller. Will be set in Debugger (by `construct_state_parents()`)

In [5]:
class State:
    """Represents the execution at a given point"""
    def __init__(self,event,abs_lineno,code,changed_vars,frame,rel_lineno,fun_name,file,fun_code):
        self.event = event
        self.abs_lineno = abs_lineno
        self.rel_lineno = rel_lineno
        self.code = code
        self.changed_vars = changed_vars
        self.frame = frame #Remove?
        self.fun_name = fun_name
        self.file = file
        self.fun_code = fun_code
        self.parent=None

    def __str__(self):
        return 'Line: ' + str(self.abs_lineno) + ' ('+ str(self.rel_lineno)+'), ('+repr(self.event)+") " + \
               repr(self.code) + '   '+ repr(self.changed_vars) + '   ' + repr(self.fun_name) + '  ('+ repr(self.file)+')'
        # return 'Line: ' + str(self.abs_lineno) + ' ('+ str(self.rel_lineno)+'), ('+repr(self.event)+") " + \
        #        repr(self.fun_code[self.rel_lineno-1]) + '   '+ repr(self.changed_vars) + '   ' + repr(self.fun_name) + '  ('+ repr(self.file)+')'




## TimeTravelDebugger
This is class to use in a with block around the code that is to be traced. The TimeTravelDebugger class contains the tracing functionality. It
records a log of program states. The main difference to the lecture's Tracer is that instead of printing stuff a list of States is generated.
After the tracing has been turned off again, the interactive session is started by using the Debugger class. It is initialized with the traced log.

Only changed variables are recorded and save to each State
Member variables have a `'member_'`-prefix.

In [6]:
class TimeTravelDebugger(object):
    """Trace a block of code and create a log for it. Then call a Debugger object which controls the interactive session """
    def __init__(self, interactive_session=True,file=sys.stdout):
        self.original_trace_function = None
        self.file = file
        self.loglist =[]
        self.last_vars = {}
        self.interactive_session=interactive_session

    def changed_vars(self, new_vars):
        # only changed variables are recorded
        changed = {}
        variables = new_vars.copy()
        for var in new_vars:
            if var=="self":
                try:
                    val =eval(var,globals(),variables)
                    if callable(type(val)):
                        member_vars = val.__dict__.copy()
                        for member_var,val in member_vars.items():
                            variables["member_"+member_var]=val
                except Exception:
                    pass

        for var_name in variables:
            if (var_name not in self.last_vars or
                    self.last_vars[var_name] != variables[var_name]):
                changed[var_name] = variables[var_name]
        self.last_vars = variables.copy()
        return changed

    def log(self, *objects, sep=' ', end='\n', flush=False):
        """Like print(), but always sending to file given at initialization,
           and always flushing"""
        current_state = State(objects[0], objects[1],objects[2],objects[3],objects[4],objects[5],objects[6],objects[7],objects[8])
        self.loglist.append(current_state)


    def traceit(self, frame, event, arg):
        """Tracing function."""
        self.log_state(frame, event, arg)

    def log_state(self, frame, event, arg):

        changes = self.changed_vars(frame.f_locals)

        source,function_start = inspect.getsourcelines(frame.f_code)
        path=inspect.getsourcefile(frame)
        _, file = os.path.split(path)
        relative_lineno = frame.f_lineno-function_start + 1
        current_line = source[relative_lineno - 1]
        func_code = source

        if event == 'call':
            self.log(event, frame.f_lineno, frame.f_code.co_name, changes, frame, relative_lineno,frame.f_code.co_name,file,func_code)

        if event == 'line':

            self.log(event, frame.f_lineno, current_line, changes, frame, relative_lineno,frame.f_code.co_name,file,func_code)

        if event == 'return':
            self.log(event, frame.f_lineno, repr(arg), changes, frame, relative_lineno,frame.f_code.co_name,file,func_code)
            self.last_vars = {}  # Delete 'last' variables


    def _traceit(self, frame, event, arg):
        """Internal tracing function."""
        if frame.f_code.co_name == '__exit__':
            # Do not trace our own _exit_() method
            pass
        else:
            self.traceit(frame, event, arg)
        return self._traceit

    def __enter__(self):
        """Called at begin of `with` block. Turn tracing on."""
        self.original_trace_function = sys.gettrace()
        sys.settrace(self._traceit)
        return self

    def __exit__(self, tp, value, traceback):
        """Called at end of `with` block. Turn tracing off. Then start interactive session."""
        sys.settrace(self.original_trace_function)
        if self.interactive_session:
            debugger = Debugger(self.loglist)
            #self.print_log()
            debugger.start()


    def print_log(self):
        for log in self.loglist:
            print(log)

    def get_log(self):
        return self.loglist

## Debugger
This class manages the actual interaction. It stores and navigates the program states.
The command dispatcher is essentially the same as in the lecture.
About the attributes:
- `callstack_depth` is used to store the position in the callstack when using up or down commands. A value of -1 indicates there has been no such command since the last navigation command
- `interact` controls whether the Debugger will wait for console commands
- `variables` represents the variables for the current step/state
- `execution_log` is a list of all States
- `print_debugger_status` prints a representation of the current state
- `help_command` help view for the commands, same as lecture /R2/
- `quit_command` ends the interactive session /R1/

/R3/ is distributed over all the the command methods.

In [7]:
IMPLEMENTED.add("/R1/")
IMPLEMENTED.add("/R2/")

In [8]:
class Debugger(object):
    """This is the core of the interactive debugger. Handles all commands."""
    def __init__(self,execution_log,interactive_session=True):

        self.current_state = None
        self.callstack_depth = -1
        self.current_step = 0
        self.breakpoints = dict()
        self.watchpoints = dict()
        self.interact = interactive_session
        self.variables = {}
        self.interactive_session = interactive_session
        self.execution_log=execution_log


    def start(self):
        """Initialize values and start the interaction."""
        if len(self.execution_log)<1:
            print("Log is empty")
            return
        self.current_state=self.execution_log[0]
        self.construct_state_parents()
        if self.interact:
            self.step_command()
            self.interaction_loop()


    def interaction_loop(self):
        """Takes command inputs."""
        while self.interact:
            command = input("(debugger) ")
            self.execute(command)

    def commands(self):
        cmds = [method.replace('_command', '')
                for method in dir(self.__class__)
                if method.endswith('_command')]
        cmds.sort()
        return cmds


    def command_method(self, command):
        if command.startswith('#'):
            return None  # Comment

        possible_cmds = [possible_cmd for possible_cmd in self.commands()
                         if possible_cmd==command]
        if len(possible_cmds) != 1:
            self.help_command(command)
            return None

        cmd = possible_cmds[0]
        return getattr(self, cmd + '_command')

    def execute(self, command):
        sep = command.find(' ')
        if sep > 0:
            cmd = command[:sep].strip()
            arg = command[sep + 1:].strip()
        else:
            cmd = command.strip()
            arg = ""

        method = self.command_method(cmd)
        if method:
            method(arg)


    def print_debugger_status(self):
        """Print the currently viewed line of the debugger"""
        event = self.current_state.event

        if event == 'call':
            self.log("This is a call event, which should never be printed")

        if event == 'line':
            current_line = self.current_state.code
            self.log(repr(self.current_state.abs_lineno) + '    ' + current_line)
        if event == 'return':
            self.log("This is a return event, which should never be printed")

    def log(self, *objects, sep=' ', end='\n', flush=False):
        """Like print(), but always sending to file given at initialization,
           and always flushing"""
        print(*objects, sep=sep, end=end, file=sys.stdout, flush=True)

    def help_command(self, command=""):
        """Give help on given command. If no command is given, give help on all"""
        if command:
            possible_cmds = [possible_cmd for possible_cmd in self.commands()
                             if possible_cmd.startswith(command)]

            if len(possible_cmds) == 0:
                print(f"Unknown command {repr(command)}. Possible commands are:")
                possible_cmds = self.commands()
            elif len(possible_cmds) > 1:
                print(f"Ambiguous command {repr(command)}. Possible expansions are:")
        else:
            possible_cmds = self.commands()

        for cmd in possible_cmds:
            method = self.command_method(cmd)
            print(f"{cmd:10} -- {method.__doc__}")


    def quit_command(self,arg=""):
        if arg:
            self.log("Invalid argument")
            return
        """End the debugger session"""
        self.interact=False
        self.log("Quit debugger session")

## Internal navigation functions
Below are functions that allow the Debugger to navigate the execution States backwards and forwards in single steps.
These are not commands in themselves.
Each direction has a method to jump next (or previous) execution no matter its event-type, and one which only jumps to "line"-events

In [9]:
class Debugger(Debugger):
    def step(self):
        """Advances the execution by one step and calculates the variables. Returns false if at the end"""
        if self.current_step<len(self.execution_log)-1:
            self.current_step += 1
            self.current_state = self.execution_log[self.current_step]
            self.construct_vars()
            self.callstack_depth=-1
            return True
        else:
            return False

    def step_to_line(self):
        """Advances the execution to the next 'line' event"""
        self.callstack_depth=-1
        init_step = self.current_step
        while self.current_step<len(self.execution_log)-1:
            self.current_step += 1
            self.current_state = self.execution_log[self.current_step]
            if self.current_state.event=="line":
                self.variables=self.construct_vars_at(self.current_step)
                return True
        self.current_step=init_step
        self.current_state=self.execution_log[self.current_step]
        return False

    def backstep_to_line(self):
        """Steps backwards in the execution to the previous 'line' event"""
        init_step = self.current_step
        while self.current_step>1:
            self.current_step -= 1
            self.current_state = self.execution_log[self.current_step]
            if self.current_state.event=="line":
                self.variables=self.construct_vars_at(self.current_step)
                self.callstack_depth=-1
                return True
        self.current_step=init_step
        self.current_state=self.execution_log[self.current_step]
        return False


    def step_back(self):
        """Steps backwards in the execution"""
        if self.current_step> 0:
            self.current_step -= 1
            self.current_state = self.execution_log[self.current_step]
            self.callstack_depth=-1
            return True
        else:
            return False


## Callstack
Below are the callstack related methods.
- `construct_state_parents()` This is called once at the beginning to set the parent attribute of states
- `construct_current_callstack()` This uses the parent attribute to return a list of callers up to the current_state, which represent the callstack
- `call_depth` This returns the depth of the current state in the callstack.
- `where_command()` /R14/
- `up_command()` /R15/
- `down_command()` /R15/

We didn't manage to make the very first frame accessible, so the top entry in the call_stack is more or less 'fake' and cannot be accessed

In [10]:
IMPLEMENTED.add("/R14/")
IMPLEMENTED.add("/R15/")

In [11]:
class Debugger(Debugger):
    def construct_state_parents(self):
        current_parent = None
        callpoints = list()
        current_depth = 0
        for i in range(0,len(self.execution_log)):
            currentexecution = self.execution_log[i]
            currentexecution.parent = current_parent

            if currentexecution.event == 'call':
                current_parent = self.execution_log[i]
                if self.execution_log[i].parent is not None :
                    current_caller = self.execution_log[i-1]
                    callpoints.append(current_caller)
                    current_parent = current_caller
                else:
                    callpoints.append(current_parent)

                current_depth = current_depth + 1

            elif currentexecution.event == 'return':
                current_depth = current_depth - 1
                caller = callpoints[current_depth]
                current_parent = caller.parent


    def construct_current_callstack(self):
        """Construct callstack from current_state to the start by iterating over the parent attribute"""
        current_callstack = list()
        current_caller = self.current_state

        while current_caller is not None :
            if current_caller.parent!=current_caller:
                current_callstack.append(current_caller)
                current_caller = current_caller.parent

        current_callstack.reverse()
        return current_callstack


    def call_depth(self,state=None):
        """Returns the depth of the current (or given step)"""
        if not state:
            state = self.current_state
        depth = 0
        while state.parent:
            state = state.parent
            depth += 1
        return depth


    def where_command(self,arg=""):
        """Print the current call stack or <number> many leading and trailing lines if given"""
        current_callstack = self.construct_current_callstack()
        limit = len(current_callstack) #Just has to be large enough in case no arg is given
        if arg:
            if arg.isdigit():
                limit = int(arg)
            else:
                self.log("Argument has to be integer")
                return

        if self.callstack_depth == -1:
            self.callstack_depth = len(current_callstack)-1

        # for i in range(0,len(current_callstack)):
        for i in range(max(0,self.callstack_depth-limit),min(len(current_callstack),self.callstack_depth+limit+1)):
            call = current_callstack[i]
            if i == self.callstack_depth:
                self.log('>> File "'+ str(call.file)+'", Line '+ str(call.abs_lineno)  + ', in '+str(call.fun_name))
            else:
                if call:
                    if call.parent:
                        # 'Fake' first call frame
                        self.log('File "'+str(call.file)+'", Line '+ str(call.abs_lineno)  + ', in '+ str(call.fun_name))
                    else:
                        self.log('File "'+str(call.file)+'", Line '+ str(call.abs_lineno)  + ', in '+ "<module>")



    def up_command(self,arg=""):
        if arg:
            self.log("Invalid argument")
            return
        """Move up the call stack towards the callers"""
        current_callstack = self.construct_current_callstack()
        if self.callstack_depth==-1:
            self.callstack_depth=len(current_callstack)-1
        if self.callstack_depth>1:
            self.callstack_depth -= 1
        else:
            self.log("Already at the highest layer")
            return

        call = current_callstack[self.callstack_depth]

        #print the function definition and mark the line of the function call
        for i in range(0,len(call.fun_code)):
            if call.rel_lineno-1==i:
                self.log(">>"+call.fun_code[i])
            else:
                self.log(call.fun_code[i])


    def down_command(self,arg=""):
        if arg:
            self.log("Invalid argument")
            return
        """Move down the call stack towards the callees"""
        current_callstack=self.construct_current_callstack()
        if self.callstack_depth==-1:
            self.callstack_depth=len(current_callstack)-1
        if self.callstack_depth==len(current_callstack)-1:
            self.log("Already on lowest layer of call stack")
        else:
            self.callstack_depth+=1
            call = current_callstack[self.callstack_depth]

            #print the function definition and mark the line of the function call
            for i in range(0,len(call.fun_code)):
                if call.rel_lineno-1==i:
                    self.log(">>"+call.fun_code[i])
                else:
                    self.log(call.fun_code[i])


## Variable calculation
Below are methods to construct the variables for states from the changed_vars attributes of the previous states, which should constitute a may-have.
One of these methods is used after every step through the execution log.
- `construct_vars()` This constructs and sets the variables but only works when advancing forward in individual steps
- `construct_vars_at()` This constructs and returns the variables for any given step
- `backconstruct_vars()` This constructs and sets the variables for the current state even when going backwards (ie from the changed_vars of all relevant preceeding states)


In [12]:
class Debugger(Debugger):
    def backconstruct_vars(self):
        """Construct the variables for the current step"""
        backconstructed_vars = dict()
        if self.current_step>0:
            num_subroutines = 0
            for i in range(0,self.current_step):
                current_execution = self.execution_log[self.current_step-(i+1)]
                if current_execution.event == 'return':
                    num_subroutines  +=1
                elif current_execution.event == 'call':
                    if num_subroutines == 0: break
                    num_subroutines -=1
                elif num_subroutines==0:
                    changed_vars = current_execution.changed_vars.copy()
                    changed_vars.update(backconstructed_vars)
                    backconstructed_vars = changed_vars

            self.variables = backconstructed_vars
            return  True
        else:
            self.variables=self.execution_log[0].changed_vars.copy()
            return False

    def construct_vars_at(self,target_step):
        """Constructs the variables at any execution step"""
        constructed_vars = dict()
        if target_step>0:
            num_subroutines = 0
            for i in range(0,target_step+1):
                current_execution = self.execution_log[target_step-i]
                if current_execution.event == 'return':
                    if i>0:
                        num_subroutines  +=1
                elif current_execution.event == 'call':
                    if num_subroutines == 0:
                        changed_vars = current_execution.changed_vars.copy()
                        changed_vars.update(constructed_vars)
                        constructed_vars = changed_vars
                        break
                    num_subroutines -=1
                elif num_subroutines==0:
                    changed_vars = current_execution.changed_vars.copy()
                    changed_vars.update(constructed_vars)
                    constructed_vars = changed_vars
        else:
            self.variables=self.execution_log[0].changed_vars.copy()
        return constructed_vars

    def construct_vars(self):
        """Constructs variables at the current step when advancing one step"""
        event = self.current_state.event
        if event == 'return':
            self.variables.clear()
        elif event == 'call':
            self.variables = self.current_state.changed_vars.copy()
        else:
            self.variables.update(self.current_state.changed_vars)

## Navigation commands 1
Below are some of the navigation methods implementing the corresponding commands. These all work in very similar ways.
The latter four use the depth in the callstack to reliably find the start and end of functions to be iterated through or over.
- `step_command()` Execute the current line and step to the next /R4/
- `backstep_command()` Unexecute and go to the previous line /R5/
- `next_command()` Step over function calls /R6/
- `previous_command()` Step over function calls backwards /R7/
- `finish_command()` Execute until the end of the current function /R8/
- `start_command()` Execute backwards to the start of the current function /R9/

In [13]:
IMPLEMENTED.add("/R4/")
IMPLEMENTED.add("/R5/")
IMPLEMENTED.add("/R6/")
IMPLEMENTED.add("/R7/")
IMPLEMENTED.add("/R8/")
IMPLEMENTED.add("/R9/")

In [14]:
class Debugger(Debugger):
    def step_command(self, arg=""):
        if arg:
            self.log("Invalid argument")
            return
        """Execute up to the next line"""
        self.step_to_line()
        self.check_watchpoints()
        self.print_debugger_status()


    def backstep_command(self,arg=""):
        if arg:
            self.log("Invalid argument")
            return
        """Step to the previous executed line"""
        self.backstep_to_line()
        self.check_watchpoints()
        self.print_debugger_status()


    def next_command(self,arg=""):
        if arg:
            self.log("Invalid argument")
            return
        """Step over function calls going to the next line"""
        if self.current_step<len(self.execution_log)-1:
            if self.execution_log[self.current_step+1].event=="call":
                self.step()
                fun_name = self.current_state.fun_name
                depth = self.call_depth()
                while not (self.current_state.event == 'return' and self.current_state.fun_name == fun_name and self.call_depth()-1 ==depth):
                    self.step()
                    self.variables = self.construct_vars_at(self.current_step)
                    if self.has_breakpoint():
                        self.check_watchpoints()
                        self.print_debugger_status()
                        return

        self.step_to_line()
        self.check_watchpoints()
        self.print_debugger_status()

    def previous_command(self,arg=""):
        """Step over function calls going to the previous line"""
        if arg:
            self.log("Invalid argument")
            return
        if self.current_step>1:
            fun_name = self.current_state.fun_name
            depth = self.call_depth()
            if self.execution_log[self.current_step-1].event=="return":
                self.backstep_to_line()
                while not (self.current_state.fun_name == fun_name and self.call_depth()==depth):
                    if self.has_breakpoint(True):
                        break
                    self.backstep_to_line()
            else:
                self.backstep_to_line()

        self.check_watchpoints()
        self.print_debugger_status()


    def start_command(self,arg=""):
        """Execute backwards until a function start"""
        if arg:
            self.log("Invalid argument")
            return
        fun_name = self.current_state.fun_name
        depth = self.call_depth()
        while not (self.current_state.fun_name == fun_name and self.current_state.event=="call" and self.call_depth()+1==depth):
            self.step_back()
            self.variables = self.construct_vars_at(self.current_step)
            if self.has_breakpoint(True):
                self.check_watchpoints()
                self.print_debugger_status()
                return
        self.step_to_line()
        self.check_watchpoints()
        self.print_debugger_status()


    def finish_command(self,arg=""):
        """Execute until return of current function"""
        if arg:
            self.log("Invalid argument")
            return
        fun_name = self.current_state.fun_name
        depth = self.call_depth()
        while not (self.current_state.fun_name == fun_name and self.current_state.event=="return" and self.call_depth()==depth):
            self.step()
            self.variables = self.construct_vars_at(self.current_step)
            if self.has_breakpoint():
                self.check_watchpoints()
                self.print_debugger_status()
                return

        self.backstep_to_line()
        self.check_watchpoints()
        self.print_debugger_status()

## Navigation commands 2
Below are more navigation methods implementing the corresponding commands. These two work very similarly.
- `continue_command()` Execute forwards until either the program finishes or a breakpoint hits /R12/
- `reverse_command()` Execute backwards until either the program start or a breakpoint hits /R13/

In [15]:
IMPLEMENTED.add("/R12/")
IMPLEMENTED.add("/R13/")

In [16]:
class Debugger(Debugger):
    def continue_command(self, arg=""):
        """ Continue execution forward until a breakpoint is hit, or the program finishes"""
        if arg:
            self.log("Invalid argument")
            return
        stepping = True
        while stepping:
            stepped = self.step_to_line()
            if (not stepped) or self.has_breakpoint():
                stepping = False

        self.check_watchpoints()
        self.print_debugger_status()

    def reverse_command(self, arg=""):
        """Continue execution backward until a breakpoint is hit, or the program starts"""
        if arg:
            self.log("Invalid argument")
            return
        stepping = True
        while stepping:
            stepped = self.backstep_to_line()
            if not stepped or self.has_breakpoint(True):
                stepping = False
        self.check_watchpoints()
        self.print_debugger_status()



## List command
Below is this implementation of the list command (and helper functions thereof)
- `list_command()` Lists the specified number of lines of code in the current function /R16/
- `lines_above()` Helper function returning a list of lines above (of specified length)
- `lines_below()` Helper function returning a list of lines below (of specified length)

In [17]:
IMPLEMENTED.add("/R16/")

In [18]:
class Debugger(Debugger):
    def lines_above(self,num_lines):
        """Returns num_lines lines of code of the current function before the current step"""
        surrounding_lines = list()
        fun_code = self.current_state.fun_code
        if fun_code:
            index = self.current_state.rel_lineno -1

            for i in range(max(0,index-num_lines),index):
                surrounding_lines.append(fun_code[i])
        return surrounding_lines


    def lines_below(self,num_lines):
        """Returns num_lines lines of code of the current function after the current step"""
        surrounding_lines = list()
        fun_code = self.current_state.fun_code
        if fun_code:
            index = self.current_state.rel_lineno - 1
            if index<len(fun_code)-1:
                for i in range(index+1, min(len(fun_code),index+num_lines+1)):
                    surrounding_lines.append(fun_code[i])
        return surrounding_lines


    def list_command(self,args=""):
        """Print the source code around the current line
        'list' prints two lines above and below
        'list <number>' prints <number> lines above and below
        'list <above> <below>' prints <above> lines above and <below> lines below"""
        arg_iterator = map(int, re.findall(r'\d+', args))

        if args!="" and re.match('^[0-9 |' ']*$',args):

             parsed_args = list()

             for parsed_arg in arg_iterator:
                 parsed_args.append(parsed_arg)
             surrounding_lines = list()

             if len(parsed_args) == 0:

                 surrounding_lines = self.lines_above(2)
                 surrounding_lines = surrounding_lines + [str(self.current_state.abs_lineno)+(">> "+ str(self.current_state.code))]
                 surrounding_lines = surrounding_lines + self.lines_below(2)
                 for surrounding_line in surrounding_lines:
                      print(str(surrounding_line))

             elif len(parsed_args) == 1:
                 surrounding_lines = surrounding_lines + (self.lines_above(parsed_args[0]))
                 surrounding_lines = surrounding_lines + [str(self.current_state.abs_lineno)+(">> " + str(self.current_state.code))]
                 surrounding_lines = surrounding_lines + (self.lines_below(parsed_args[0]))
                 for surrounding_line in surrounding_lines:
                      print(str(surrounding_line))

             elif len(parsed_args) == 2:
                 surrounding_lines  = surrounding_lines + (self.lines_above(parsed_args[0]))
                 surrounding_lines = surrounding_lines + [str(self.current_state.abs_lineno)+(">> " + str(self.current_state.code))]
                 surrounding_lines  = surrounding_lines + (self.lines_below(parsed_args[1]))
                 for surrounding_line in surrounding_lines:
                      print(str(surrounding_line))
             else:
                self.log("Wrong usage: list <above> <below> :Print <above> lines before and <below> lines after the current line")

        elif args.replace(" ","")== "":
             surrounding_lines = self.lines_above(2)
             surrounding_lines = surrounding_lines + [str(self.current_state.abs_lineno)+(">> "+ str(self.current_state.code))]
             surrounding_lines = surrounding_lines + self.lines_below(2)
             for surrounding_line in surrounding_lines:
                  print(str(surrounding_line))
        else:
            self.log("Wrong usage: list <above> <below> :Print <above> lines before and <below> lines after the current line")

## Until and backuntil commands
Below is this implementation of the until and backuntil commands (and helper functions thereof)
- `until_command()` Executes the program forward up until the point specified in the parameters. Parses the arguments and calls the corresponding helper functions. /R10/
- `backuntil_command()` Executes the program backwards up to the point specified in the parameters. Parses the arguments and calls the corresponding helper functions. /R11/
- `until_function()` Helper. Called when executing forwards until a specified function is called.
- `backuntil_function()` Helper. Called when executing backwards until a specified function returns.
- `until_line()` Helper. Called when executing forwards until a line greater than the specified one is called.
- `backuntil_line()` Helper. Called when executing backwards until a line less than the specified one is called.
We implemented in the following way: We step through the executions forwards (or backwards respectively), and stop if
the condition is met in any of the following (preceding) states.

In [19]:
IMPLEMENTED.add("/R10/")
IMPLEMENTED.add("/R11/")

In [20]:
class Debugger(Debugger):
    def until_command(self,arg=""):
        """Execute forwards until a certain point.
        'until' executes until a line greater than the current line is reached
        'until <line_number>' executes until a line greater than <line_number> is reached
        'until <filename>:<line_number>' executes until line <line_number> is reached in file <filename>
        'until <function_name>' executes until function <function_name> is called
        'until <filename>:<function_name>' executes until function <function_name> declared in <filename> is called
        """
        #TODO special case no arg act as next
        if not arg:
            #no arg case
            self.until_line(self.current_state.abs_lineno,self.current_state.file)
            return

        function_name=""
        target_line=0
        sep = arg.find(':')
        if sep > 0:
            #compound arg case
            filename = arg[:sep].strip()
            secondary_arg = arg[sep+1:].strip()
            if secondary_arg.isdigit():
                target_line=int(secondary_arg)
            else: 
                function_name = secondary_arg

            if target_line > 0:
                self.until_line(target_line,filename)
            else:
                self.until_function(function_name,filename)
        else:
            #simple arg case
            if arg.isdigit():
                target_line=int(arg)
            else:
                function_name=arg

            if function_name:
                self.until_function(function_name,self.current_state.file)
            else:
                self.until_line(target_line,self.current_state.file)


    def backuntil_command(self,arg=""):
        """Execute backwards until a certain point.
        'backuntil' executes backwards until a line less than the current line is reached
        'backuntil <line_number>' executes backwards until a line less than <line_number> is reached
        'backuntil <filename>:<line_number>' executes backwards until line <line_number> is reached in file <filename>
        'backuntil <function_name>' executes backwards until function <function_name> is called
        'backuntil <filename>:<function_name>' executes backwards until function <function_name> declared in <filename> is called
        """
        if not arg:
            #no arg case
            self.backuntil_line(self.current_state.abs_lineno,self.current_state.file)
            return

        function_name=""
        target_line=0
        sep = arg.find(':')
        if sep > 0:
            #compound arg case
            filename = arg[:sep].strip()
            secondary_arg = arg[sep+1:].strip()
            if secondary_arg.isdigit():
                target_line=int(secondary_arg)
            else:
                function_name = secondary_arg
            if target_line > 0:
                self.backuntil_line(target_line,filename)
            else:
                self.backuntil_function(function_name,filename)
        else:
            #simple arg case
            if arg.isdigit():
                target_line=int(arg)
            else:
                function_name=arg

            if function_name:
                self.backuntil_function(function_name,self.current_state.file)
            else:
                self.backuntil_line(target_line,self.current_state.file)

    def until_function(self,target_function,target_file):
        """Advance until call to target_function and in target_file"""
        stepped = True
        while not (self.current_state.fun_name == target_function and self.current_state.event=="call"
                   and self.current_state.file==target_file ):
            stepped = self.step()
            self.variables=self.construct_vars_at(self.current_step)
            if not stepped:
                break
            if self.has_breakpoint():
                self.check_watchpoints()
                self.print_debugger_status()
                return

        if stepped:
            self.backstep_to_line()
        else:
            self.backstep_to_line()
        self.check_watchpoints()
        self.print_debugger_status()


    def backuntil_function(self,target_function,target_file):
        """Run backwards until call to target_function and in target_file if given"""
        while not (self.current_state.fun_name == target_function and self.current_state.event=="call" and
                    (self.current_state.file==target_file)):
            stepped = self.step_back()
            if not stepped:
                break

            self.variables=self.construct_vars_at(self.current_step)
            if self.has_breakpoint(True):
                self.check_watchpoints()
                self.print_debugger_status()
                return

        self.step_to_line()
        self.check_watchpoints()
        self.print_debugger_status()


    def until_line(self,target_line,target_file):
        """Advance until program line is greater than target_line and in target_file if given"""
        if target_line<=0:
            self.log("Invalid line: " + str(target_line))
            return

        for i in range(self.current_step,len(self.execution_log)):
            if self.current_state.abs_lineno>target_line and self.current_state.file==target_file:
                break
            self.step_to_line()
            if self.has_breakpoint():
                break

        self.check_watchpoints()
        self.print_debugger_status()


    def backuntil_line(self,target_line,target_file):
        """Run backwards until program line is lower than target_line and in target_file"""
        if target_line<=1:
            self.log("Invalid line: " + str(target_line))
            return

        for i in range(0,self.current_step+1):
            if self.current_state.abs_lineno<target_line and self.current_state.file==target_file:
                break
            self.backstep_to_line()
            if self.has_breakpoint(True):
                break

        self.check_watchpoints()
        self.print_debugger_status()


## Breakpoint
This class represents breakpoints. Relevant information is stored in the member variables.
The break_here method determines based on its parameters whether the breakpoint hits or not.
There are three types of breakpoints represented by the type string. There are three types:
- `"line"` for line breakpoints (/R201/)
- `"func"` for function breakpoints (/R202/)
- `"cond"` for condition breakpoints (/R203/)

In [21]:
class Breakpoint:
    """Represents a breakpoint"""
    def __init__(self,type,line=None,function=None,condition=None,file=None,is_active=True):
        self.type=type
        self.line_number=line
        self.func_name=function
        self.cond_expression = condition
        self.file = file
        self.is_active=is_active
        self.alias = ""

    def break_here(self,step,log,variables,backwards=False):
        """Returns whether this breakpoints hits with the given arguments. The 'backwards' argument indicates if the
        execution happens backwards"""
        if not self.is_active:
            return False

        current_state = log[step]
        if self.type == 'line':
            return current_state.abs_lineno==self.line_number and current_state.file==self.file
        elif self.type == 'func':
            if backwards:
                if step < len(log)-1:
                    prev = log[step+1]
                    return prev.event=="return" and prev.fun_name==self.func_name
            else:
                if step > 1:
                    prev = log[step-1]
                    return prev.event=="call" and prev.fun_name==self.func_name
        elif self.type =='cond':
            if current_state.abs_lineno==self.line_number:
                try:
                    return eval(self.cond_expression,globals(),variables)
                except Exception:
                    return False

        return False

    def is_active(self):
        return self.is_active

    def enable(self):
        self.is_active=True

    def disable(self):
        self.is_active=False

    def set_alias(self,alias):
        self.alias = alias

    def __str__(self):
        if self.type == 'line':
            return f"line  {repr(self.file)}:{self.line_number}  {self.is_active}"
        elif self.type == 'func':
            return f"func  {repr(self.file)}:{self.func_name}  {self.is_active}"
        elif self.type =='cond':
            return f"cond  {repr(self.file)}:{self.line_number}  {self.is_active}  {repr(self.cond_expression)}"
        else:
            return ""


## Breakpoint related commands
### Back to Debugger
Below is the implementation of the commands related to breakpoints. The breakpoints (Breakpoint objects) are stored in the dictionary `self.breakpoints`.
The keys are their ids (which are therefore unique).
- `break_command()` Creates a function or line breakpoint /R201/,/R202/,/R203/
- `cond_command()` Creates a conditional breakpoint /R208/
- `delete_command()` Removes a breakpoint with the given id (or alias) /R205/
- `breakpoints_command()` Lists all breakpoints /R204/
- `disable_command()` Disables a breakpoint with the given id (or alias) /R206/
- `enable_command()` Enables a breakpoint with the given id (or alias) /R207/
- `alias_command()` Allows one to set names for breakpoints and address them with those names (/May-have/)
- `breakpoint_lookup()` Helper to allow breakpoints to be looked up both through their id and alias

In [22]:
IMPLEMENTED.add("/R201/")
IMPLEMENTED.add("/R202/")
IMPLEMENTED.add("/R203/")
IMPLEMENTED.add("/R204/")
IMPLEMENTED.add("/R205/")
IMPLEMENTED.add("/R206/")
IMPLEMENTED.add("/R207/")
IMPLEMENTED.add("/R208/")

In [23]:
class Debugger(Debugger):
    def break_command(self, arg=""):
        """Set a breakpoint.
        'break <line_number>' Set breakpoint in line <line_number>
        'break <function_name>' Set a breakpoint which hits when a function with the name <function_name> is entered
        'break <file_name>:<function_name>' Set a breakpoint which hits when a function <function_name> declared in <file_name> is entered"""
        if arg:
            if len(self.breakpoints)>0:
                largest_key= max(self.breakpoints.keys())
            else:
                largest_key = 0

            if arg.isdigit():
                self.breakpoints[largest_key+1]=Breakpoint("line",int(arg),None,None,self.current_state.file)
            else:
                sep = arg.find(':')
                if sep>0:
                    file_name = arg[:sep].strip()
                    func_name = arg[sep+1:].strip()
                    self.breakpoints[largest_key+1]=Breakpoint("func",None,func_name,None,file_name)
                else:
                    self.breakpoints[largest_key+1]=Breakpoint("func",None,arg,None,self.current_state.file)
        else:
            self.log("Missing argument, use as 'break <line>|<function_name>|<file_name>:<function_name>'")

    def delete_command(self, arg=""):
        """Delete breakpoint with given <breakpoint_id>|<breakpoint_name>"""
        if arg:
            breakpoint_id = self.breakpoint_lookup(arg)
            if breakpoint_id==-1:
                self.log("No such breakpoint: "+arg)
            else:
                self.breakpoints.pop(breakpoint_id)
        else:
            self.log("Missing argument, use as 'delete <breakpoint_id>'")

    def breakpoints_command(self,arg=""):
        """Display all available breakpoints"""
        if arg:
            self.log("Invalid argument")
            return
        for breakpoint in self.breakpoints:
            alias = self.breakpoints[breakpoint].alias
            if alias:
                self.log(alias+ "   " +str(self.breakpoints[breakpoint]))
            else:
                self.log(str(breakpoint)+ "   " +str(self.breakpoints[breakpoint]))

    def disable_command(self,arg=""):
        """Use as 'disable <breakpoint_id>|<breakpoint_name>'    Suspend  specified breakpoint"""
        if arg:
            breakpoint_id = self.breakpoint_lookup(arg)
            if breakpoint_id==-1:
                self.log("No such breakpoint: "+arg)
            else:
                self.breakpoints[breakpoint_id].disable()
        else:
            self.log("Missing argument, use as 'disable <breakpoint_id>'")

    def enable_command(self,arg=""):
        """Use as 'enable <breakpoint_id>|<breakpoint_name>'    Re-enable specified breakpoint"""
        if arg:
            breakpoint_id = self.breakpoint_lookup(arg)
            if breakpoint_id==-1:
                self.log("No such breakpoint: "+arg)
            else:
                self.breakpoints[breakpoint_id].enable()
        else:
            self.log("Missing argument, use as 'enable <breakpoint_id>'")

    def breakpoint_lookup(self,arg):
        """Find key of given breakpoint_id or breakpoint_name. Returns -1 if no such breakpoint exists"""
        if arg.isdigit():
            if int(arg) in self.breakpoints.keys():
                return int(arg)

        else:
            for breakpoint_id in self.breakpoints:
                if self.breakpoints[breakpoint_id].alias==arg:
                    return breakpoint_id
        return -1

    def alias_command(self,arg=""):
        """Use as 'alias <breakpoint_id> <breakpoint_name>'  Set the alias <breakpoint_name> for breakpoint <breakpoint_id> """
        if not arg:
            self.log("Requires arguments <breakpoint_id> <breakpoint_name>")

        sep = arg.find(' ')
        if sep > 0:
            breakpoint_id = arg[:sep].strip()
            alias = arg[sep+1:].strip()
            if not alias:
                self.log("Empty string not allowed as alias")
                return
            if breakpoint_id.isdigit():
                breakpoint_aliases=[bp.alias for bp in self.breakpoints.values()]
                if alias in breakpoint_aliases:
                    self.log("Alias already exists: "+ alias)
                    return
                try:
                    breakpoint = self.breakpoints[int(breakpoint_id)]
                    breakpoint.alias=alias
                except KeyError:
                    self.log("No breakpoint with id: " + breakpoint_id)
            else:
                self.log("Invalid arguments. Use as 'alias <breakpoint_id> <breakpoint_name>'")

        else:
            self.log("Invalid arguments. Use as 'alias <breakpoint_id> <breakpoint_name>'")


    def cond_command(self,arg=""):
        """Use as 'cond <line> <condition>'    Set a breakpoint at which the execution is stopped at line <line>
        if a condition <condition> is true"""
        if arg:
            sep = arg.find(' ')
            if sep>0:
                line = arg[:sep].strip()
                cond_expression = arg[sep+1:].strip()
                if not line.isdigit():
                    self.log("Faulty argument, use as 'cond <line> <condition>'")
                    return

                if len(self.breakpoints)>0:
                    largest_key= max(self.breakpoints.keys())
                else:
                    largest_key = 0

                self.breakpoints[largest_key+1]=Breakpoint("cond",int(line),None,cond_expression,self.current_state.file)
            else:
                self.log("Faulty argument, use as 'cond <line> <condition>'")
        else:
            self.log("Missing argument, use as 'cond <line> <condition>'")

## Breakpoint checking
This function checks whether any of the defined breakpoints hit at the current step.
To that end the relevant information of the current program state is passed to each breakpoint. They then compute with their
`break_here` function whether they result in a break.
It is called by the navigation commands every time the step forwards or backwards
The function argument `backwards` indicates whether the Debugger is executing backwards currently.
Part of /R20/

In [24]:
class Debugger(Debugger):
    def has_breakpoint(self,backwards=False):
        """Checks if any of the breakpoints trigger"""
        active_breakpoints = [bp for bp in self.breakpoints if self.breakpoints[bp].break_here(self.current_step,self.execution_log,self.variables,backwards)]
        return len(active_breakpoints)>0

## Watchpoints
Below is the implementation of the commands related to watchpoints. The watchpoints are stored in the dictionary `self.watchpoints`.
The keys are their ids.
- `watch_command()` Command which creates a watchpoint with a unique id for a given variable /R190/
- `unwatch_command()` Command which removes the specified watchpoint /R191/
- `print_watchpoints()` Command which prints all defined watchpoints /R192/
- `update_watchpoints()` This function checks for watchpoint changes (based on `self.variables`) and prints them correspondingly
- `check_watchpoints()` This function is called by navigation commands after each step to check if any watchpoints trigger.
If the program jsut returned from a function watchpoints are only triggered if they were changed by the returned value.
If the program did not return it just calls `update_watchpoints()`


In [25]:
IMPLEMENTED.add("/R190/")
IMPLEMENTED.add("/R191/")
IMPLEMENTED.add("/R192/")

In [26]:
class Debugger(Debugger):
    def watch_command(self,arg=""):
        """Creates a numbered watchpoint for the given variable"""
        if arg:
            variables = [var for (var, val) in self.watchpoints.values()]
            if not arg in variables:
                if len(self.watchpoints)>0:
                    largest_key= max(self.watchpoints.keys())
                else:
                    largest_key = 0
                self.watchpoints[largest_key+1]=(arg,None)

            else:
                self.log("Watchpoint already exists: "+arg)
        else:
            self.print_watchpoints()

    def unwatch_command(self,arg=""):
        """Remove a watchpoint with the given <watch_id>"""
        if arg:
            if arg.isdigit():
                try:
                    self.watchpoints.pop(int(arg))
                except KeyError:
                    self.log("No such watchpoint: "+ arg)
                self.log("Watchpoints", self.watchpoints)
            else:
                self.log("The argument has to be an integer: unwatch <watch_id>")
        else:
            self.log("Specify which watchpoint to remove: unwatch <watch_id>")
            self.log("Watchpoints", self.watchpoints)

    def update_watchpoints(self,print_changes=False):
        """Checks whether any of the watchpoints trigger and prints the corresponding output"""
        changes={}
        for watch_id in self.watchpoints:
            entry = self.watchpoints[watch_id]
            if entry[0] in self.variables:
                if self.variables[entry[0]]!=entry[1]:
                    changes[watch_id]=(entry[0],self.variables[entry[0]])
        self.watchpoints.update(changes)
        if print_changes and changes:
            changes_s= ", ".join([repr(self.watchpoints[watch_id][0])+" = "+ repr(self.watchpoints[watch_id][1])
                               for watch_id in changes])
            self.log("Changed watchpoints: " + changes_s)
        return changes

    def check_watchpoints(self):
        """Controls watchpoint updates. If the program just returned a watchpoint only triggers if that is due to
        being assigned to the return value"""
        if not self.watchpoints:
            return

        if self.current_step>0:
            if self.execution_log[self.current_step-1].event == 'return':
                var_backup = self.variables.copy()
                step = self.current_step-1
                state =self.execution_log[step]
                while not (step == 0 or (state.event=='line' and state.fun_name == self.current_state.fun_name)):
                    step -= 1
                    state = self.execution_log[step]
                if step != 0:
                    self.log(str(state))
                    self.variables = self.construct_vars_at(step)
                    self.update_watchpoints()
                    self.variables=var_backup
                    self.update_watchpoints(True)
            else:
                self.update_watchpoints(True)
        else:
            self.update_watchpoints(True)

    def print_watchpoints(self):
        """Print all current watchpoints"""
        watched_vars= ", ".join([str(watch_id) + ": "+repr(self.watchpoints[watch_id][0])
                               for watch_id in self.watchpoints])
        self.log(watched_vars)



## Print command
The print command prints currently available variables and expressions

In [27]:
IMPLEMENTED.add("/R17/")
IMPLEMENTED.add("/R18/")

In [28]:
class Debugger(Debugger):
    def print_command(self, arg=""):
        """Print an expression. If no expression is given, print all variables"""
        variables = self.variables
        if not arg:
            self.log("\n".join([f"{var} = {repr(variables[var])}" for var in variables]))
        else:
            try:
                self.log(f"{arg} = {repr(eval(arg, globals(), variables))}")
            except Exception as err:
                self.log(f"{err.__class__.__name__}: {err}")


    def get_log(self):
        return self.execution_log


# Examples
### Let's define a couple functions

In [29]:
def foo(arg):
    a = 12
    a= arg
    b = bar(21)
    x = 5
    recursion(3)
    #b = fun(a,x)
    a = 11
    return

def bar(arg):
    b = arg
    a = 7
    return a+b

def recursion(value):
    if value>0:
        recursion(value - 1)


### First some stepping and printing of variables (/R4/,/R17/,/R1/)
We can observe how the variables change

In [30]:
next_inputs(["print","step","print","step","print","quit"])
with TimeTravelDebugger():
    foo(2)

2        a = 12

arg = 2
3        a= arg

arg = 2
a = 12
4        b = bar(21)

arg = 2
a = 2
Quit debugger session


### Let's try backstep too (/R4/,/R5/,/R16/,/R18/,/R20/)
We can see the variable value alternating and show usage of an expression in print
We also the usage of an expression in print

In [31]:
next_inputs(["print","step","print","backstep","print","step","print max(a,10)","quit"])
with TimeTravelDebugger():
    foo(2)

2        a = 12

arg = 2
3        a= arg

arg = 2
a = 12
2        a = 12

arg = 2
3        a= arg

max(a,10) = 12
Quit debugger session


### Next and previous (/R6/,/R7/)
When we call step at the call to bar() we step into it, but with next and previous we can jump over it.
Backstep will also step into bar(), but from the other end. We also see all the variables are what they are supposed to be after the function call.

In [32]:
next_inputs(["step","step","step","backstep","next","print","backstep","step","previous","print","quit"])
with TimeTravelDebugger():
    foo(2)

2        a = 12

3        a= arg

4        b = bar(21)

12        b = arg

4        b = bar(21)

5        x = 5

arg = 2
a = 2
b = 28
14        return a+b

5        x = 5

4        b = bar(21)

arg = 2
a = 2
Quit debugger session


### Finish and start (/R8/,/R9/)
Executing until return and back to function start again. We also see the behavior at the end (and beginning) of a function

In [33]:
next_inputs(["finish","finish","start","start","step","step","step","finish","start","quit"])
with TimeTravelDebugger():
    foo(2)

2        a = 12

9        return

9        return

2        a = 12

2        a = 12

3        a= arg

4        b = bar(21)

12        b = arg

14        return a+b

12        b = arg

Quit debugger session


### List and until (/R10/,/R16/)
Let's display varying amounts of function code with list and move around with until

In [34]:
next_inputs(["list","list 3","list 100","until 5","list 1 0","quit"])
with TimeTravelDebugger():
    foo(2)


2        a = 12

def foo(arg):

2>>     a = 12

    a= arg

    b = bar(21)

def foo(arg):

2>>     a = 12

    a= arg

    b = bar(21)

    x = 5

def foo(arg):

2>>     a = 12

    a= arg

    b = bar(21)

    x = 5

    recursion(3)

    #b = fun(a,x)

    a = 11

    return

12        b = arg

def bar(arg):

12>>     b = arg

Quit debugger session


### Backuntil and more until(/R10/,/R11/)
This time with functions as arguments

In [35]:
next_inputs(["until bar","step","finish","step","backuntil bar","quit"])
with TimeTravelDebugger():
    foo(2)


2        a = 12

4        b = bar(21)

12        b = arg

14        return a+b

5        x = 5

12        b = arg

Quit debugger session


### Breakpoints and continue (/R12/,/R20/)
Let's put some breakpoints, show them, and observe the continue command stopping there. As expected the function breakpoint for recursion() is hit every time it calls itself.

In [36]:
next_inputs(["break 3","break recursion","breakpoints","continue","continue","continue","continue","continue","continue","quit"])
with TimeTravelDebugger():
    foo(2)


2        a = 12

1   line  '<ipython-input-29-ae4cb03b8ced>':3  True
2   func  '<ipython-input-29-ae4cb03b8ced>':recursion  True
3        a= arg

17        if value>0:

17        if value>0:

17        if value>0:

17        if value>0:

9        return

Quit debugger session


### Breakpoints, alias, and reverse (/R20/,/R13/,/R17/,and may-have)
Let's define a conditional breakpoint, play around with the name, and display reverse stopping.
As one can see, the break is accessible (disabling and enabling it) through it's alias

In [37]:
next_inputs(["finish","cond 4 a==2","alias 1 cond_break","breakpoints","disable cond_break","breakpoints","enable cond_break","reverse","print a","quit"])
with TimeTravelDebugger():
    foo(2)


2        a = 12

9        return

cond_break   cond  '<ipython-input-29-ae4cb03b8ced>':4  True  'a==2'
cond_break   cond  '<ipython-input-29-ae4cb03b8ced>':4  False  'a==2'
4        b = bar(21)

a = 2
Quit debugger session


In [38]:
def deep1(arg):
    #first layer
    arg+=1
    erg = deep2(arg)
    return erg

def deep2(arg):
    #second layer
    arg+=2
    return deep3(arg)

def deep3(arg):
    #third layer
    arg+=3
    return arg

###  Callstack navigation(/R14/,/R15/)
Let's step into a function and use where, up and down to navigate the callstack

In [39]:
next_inputs(["where","step","step","where","up","where","down","step","step","where","quit"])
with TimeTravelDebugger():
    deep1(1)

3        arg+=1

File "<ipython-input-38-18e0618e5b2e>", Line 1, in <module>
>> File "<ipython-input-38-18e0618e5b2e>", Line 3, in deep1
4        erg = deep2(arg)

9        arg+=2

File "<ipython-input-38-18e0618e5b2e>", Line 1, in <module>
File "<ipython-input-38-18e0618e5b2e>", Line 4, in deep1
>> File "<ipython-input-38-18e0618e5b2e>", Line 9, in deep2
def deep1(arg):

    #first layer

    arg+=1

>>    erg = deep2(arg)

    return erg

File "<ipython-input-38-18e0618e5b2e>", Line 1, in <module>
>> File "<ipython-input-38-18e0618e5b2e>", Line 4, in deep1
File "<ipython-input-38-18e0618e5b2e>", Line 9, in deep2
def deep2(arg):

    #second layer

>>    arg+=2

    return deep3(arg)

10        return deep3(arg)

14        arg+=3

File "<ipython-input-38-18e0618e5b2e>", Line 1, in <module>
File "<ipython-input-38-18e0618e5b2e>", Line 4, in deep1
File "<ipython-input-38-18e0618e5b2e>", Line 10, in deep2
>> File "<ipython-input-38-18e0618e5b2e>", Line 14, in deep3
Quit debugger session


###  Watchpoints (/R19/)

We set watchpoints for 'a' and 'something', move through the program a bit to see the print, and then unwatch the second watchpoint.

In [40]:
next_inputs(["watch a","watch something","watch","step","step","finish","print","unwatch 2","watch","quit"])
with TimeTravelDebugger():
    foo(2)

2        a = 12

1: 'a', 2: 'something'
Changed watchpoints: 'a' = 12
3        a= arg

Changed watchpoints: 'a' = 2
4        b = bar(21)

Changed watchpoints: 'a' = 11
9        return

arg = 2
a = 11
b = 28
x = 5
Watchpoints {1: ('a', 11)}
1: 'a'
Quit debugger session


###  Help and missing arguments (/R2/,/R3/)
Let's try the help command and try an undefined command. Let's also use faulty arguments with some commands.

In [41]:
next_inputs(["help","unknown","delete 2","list a","print len(2)","until 0","cond notaline a==2","quit"])
with TimeTravelDebugger():
    foo(2)


2        a = 12

alias      -- Use as 'alias <breakpoint_id> <breakpoint_name>'  Set the alias <breakpoint_name> for breakpoint <breakpoint_id> 
backstep   -- None
backuntil  -- Execute backwards until a certain point.
        'backuntil' executes backwards until a line less than the current line is reached
        'backuntil <line_number>' executes backwards until a line less than <line_number> is reached
        'backuntil <filename>:<line_number>' executes backwards until line <line_number> is reached in file <filename>
        'backuntil <function_name>' executes backwards until function <function_name> is called
        'backuntil <filename>:<function_name>' executes backwards until function <function_name> declared in <filename> is called
        
break      -- Set a breakpoint.
        'break <line_number>' Set breakpoint in line <line_number>
        'break <function_name>' Set a breakpoint which hits when a function with the name <function_name> is entered
        'break <file

## UITracer
This is a slightly different version of the tracing functionality of the TimeTravelDebugger. It also doesn't start a debugging session,
as it is intended to trace for the UI.

In [42]:
class UITracer(object):
    """Trace a block of code and create a log for it. Then call a Debugger object which controls the interactive session """
    def __init__(self, interactive_session=True,file=sys.stdout):
        self.original_trace_function = None
        self.file = file
        self.loglist =[]
        self.last_vars = {}
        self.interactive_session=interactive_session


    def log(self, *objects, sep=' ', end='\n', flush=False):
        """Like print(), but always sending to file given at initialization,
           and always flushing"""
        current_state = State(objects[0], objects[1],objects[2],objects[3],objects[4],objects[5],objects[6],objects[7],objects[8])
        self.loglist.append(current_state)


    def traceit(self, frame, event, arg):
        """Tracing function."""
        self.log_state(frame, event, arg)

    def log_state(self, frame, event, arg):
        source,function_start = inspect.getsourcelines(frame.f_code)

        for i in range(0,len(source)):
            source[i]=str(function_start+i)+ " " + source[i]

        path=inspect.getsourcefile(frame)
        _, file = os.path.split(path)
        relative_lineno = frame.f_lineno-function_start + 1
        current_line = source[relative_lineno - 1]
        func_code = source

        if event == 'call':
            self.log(event, frame.f_lineno, frame.f_code.co_name, frame.f_locals.copy(), frame, relative_lineno,frame.f_code.co_name,file,func_code)

        if event == 'line':
            self.log(event, frame.f_lineno, current_line, frame.f_locals.copy(), frame, relative_lineno,frame.f_code.co_name,file,func_code)

        if event == 'return':
            self.log(event, frame.f_lineno, repr(arg), frame.f_locals.copy(), frame, relative_lineno,frame.f_code.co_name,file,func_code)


    def _traceit(self, frame, event, arg):
        """Internal tracing function."""
        if frame.f_code.co_name == '__exit__':
            # Do not trace our own _exit_() method
            pass
        else:
            self.traceit(frame, event, arg)
        return self._traceit

    def __enter__(self):
        """Called at begin of `with` block. Turn tracing on."""
        self.original_trace_function = sys.gettrace()
        sys.settrace(self._traceit)
        return self

    def __exit__(self, tp, value, traceback):
        """Called at end of `with` block. Turn tracing off. Then start interactive session."""
        sys.settrace(self.original_trace_function)


    def print_log(self):
        for log in self.loglist:
            print(log)

    def get_log(self):
        return self.loglist













In [43]:
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

outputstream = widgets.Output()
out = widgets.HTML(
                    value='<h3> dasdadsdssad </h3>',
                    placeholder='',
                    description='',
                )
def whichNumber(number):
    global out
    outputstream.append_stdout(str(number))
    out.value = str(number)

display(out)
int_range = widgets.IntSlider(min=1,max=10,value=1,description='Value')
interact(whichNumber,number=int_range)

HTML(value='<h3> dasdadsdssad </h3>', placeholder='')

interactive(children=(IntSlider(value=1, description='Value', max=10, min=1), Output()), _dom_classes=('widget…

<function __main__.whichNumber(number)>

In [44]:

with TimeTravelDebugger(False) as ttd:
    foo(1)
execution_log = ttd.get_log()
debugger = Debugger(execution_log=execution_log,interactive_session=False)
debugger.start()

TypeError: foo() missing 1 required positional argument: 'arg'

In [ ]:
def create_expanded_button(description, button_style):
    return widgets.Button(description=description, button_style=button_style, layout=widgets.Layout(height='auto', width='auto'))

play = widgets.Play(
#     interval=10,
    value=0,
    min=0,
    max=100,
    step=1,
    description="Press play",
    disabled=False
)
slider = widgets.IntSlider()
widgets.jslink((play, 'value'), (slider, 'value'))
tester = widgets.HBox([play, slider])


top_left_button = create_expanded_button("Top left", 'info')
top_right_button = create_expanded_button("Top right", 'success')
bottom_left_button = create_expanded_button("Bottom left", 'danger')
bottom_right_button = create_expanded_button("Bottom right", 'warning')

top_left_text = widgets.IntText(description='Top left', layout=widgets.Layout(width='auto', height='auto'))
top_right_text = widgets.IntText(description='Top right', layout=widgets.Layout(width='auto', height='auto'))
bottom_left_slider = widgets.IntSlider(description='Bottom left', layout=widgets.Layout(width='auto', height='auto'))
bottom_right_slider = widgets.IntSlider(description='Bottom right', layout=widgets.Layout(width='auto', height='auto'))

html_code = list()
functions = list()
lines = list()

for execution in execution_log:
    if execution.fun_code not in functions:
        current_function = list()
        for line in execution.fun_code:
            current_function.append(line)
        functions.append(current_function)

for function in functions:
    for line in function:
        if 'def ' in line:
            my_html = widgets.HTML(
                    value='<h3>'+ line + '</h3>',
                    placeholder='',
                    description='',
                )
            html_code.append(my_html)
        else:
            my_html = widgets.HTML(
                    value='<h4>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;'+ line + '</h4>',
                    placeholder='',
                    description='',
                )
            html_code.append(my_html)

def whichLine(number):
    global html_code
    html_code[number].value='>> ' + html_code[number.value]

int_range = widgets.IntSlider(min=1,max=len(execution_log),value=1,description='Execution Step')
interact(whichLine,number=int_range)

box = widgets.VBox(html_code)
box.observe(whichLine)

header_button = create_expanded_button('Header', 'success')
left_button = create_expanded_button('Left', 'info')
center_button = create_expanded_button('Center', 'warning')
right_button = create_expanded_button('Right', 'info')
footer_button = create_expanded_button('Footer', 'success')

widgets.AppLayout(header=tester,
          center=box,
          right_sidebar=right_button,
          pane_widths=[2, 0.5, 1],
          pane_heights=[1, 17, 1])

In [ ]:
with UITracer() as tr:
    foo(1)

In [ ]:
log = tr.get_log()

current_parent = None
callpoints = list()
current_depth = -1
for i in range(0,len(log)):
    currentexecution = log[i]
    currentexecution.parent = current_parent

    if currentexecution.event == 'call':
        current_parent = log[i]
        if log[i].parent is not None :
            current_caller = log[i-1]
            callpoints.append(current_caller)
            current_parent = current_caller
        else:
            callpoints.append(current_parent)

        current_depth = current_depth + 1

    elif currentexecution.event == 'return':
        current_depth = current_depth - 1
        caller = callpoints[current_depth]
        current_parent = caller.parent

current_callstack = None
callstack_layer = -1

def construct_current_callstack(current_state):
    global callstack_layer,current_callstack,current_depth
    current_callstack = list()
    current_caller = current_state

    while current_caller is not None :
        if current_caller.parent!=current_caller:
            current_callstack.append(current_caller)
            current_caller = current_caller.parent

    current_callstack.reverse()
    callstack_layer = len(current_callstack)-1

    if current_depth==-1:
        current_depth = callstack_layer
    return current_callstack

a = widgets.IntSlider(min=0,max=len(log)-1,value=1,description='step')
b = widgets.IntSlider(description='b')
c = widgets.IntSlider(description='c')

add_breakpoint_input = widgets.Text(
    value='',
    placeholder='Enter: <name of function> <line> and other breakpoint information',
    description='',
    disabled=False
)
remove_breakpoint_input = widgets.Text(
    value='',
    placeholder='Enter <name of function> <line> of breakpoint to remove',
    description='',
    disabled=False
)
# def state_display(number):
#     log = debugger.get_log()
#     fun_code = log[number].fun_code.copy()
#     fun_code[log[number].rel_lineno-1] = ">>"+fun_code[log[number].rel_lineno-1]
#
#     for line in fun_code:
#         print(line)

def code_display(step):
    global breakpoints
    #fun_code = "\n".join([line for line in log[number].fun_code])
    fun_code = log[step].fun_code.copy()
    fun_code[log[step].rel_lineno-1] = ">>"+fun_code[log[step].rel_lineno-1]

    for line in fun_code:
        print('  '+ str(line))

breakpoints = set()
watchpoints = set()

def var_display(step):
    print(log[step].changed_vars)
    print('\nBreakpoints: ')
    for breakpoint in breakpoints:
        print('Absolute Line: '+ str(breakpoint.line_number) + ', Function: '+str(breakpoint.func_name))
    #print(log[number].changed_vars)

def callstack_display(step,layer):
    global callstack_layer,current_depth

    print('\nCurrent callstack:')
    current_callstack = construct_current_callstack(log[step])
    if current_depth==-1:
        current_depth = len(current_callstack)-1
    print('\nCurrent Depth:' + str(current_depth))

    if callstack_layer == -1:
        callstack_layer = len(current_callstack) -1
    elif callstack_layer>0:
        callstack_layer-=1
    else:
        print('Already at highest layer of call stack')

    for i  in range(0,len(current_callstack)):
        caller = current_callstack[i]
        if i == current_depth:
            print('>> File "'+str(caller.file)+', Line '+str(caller.rel_lineno))
        else:
            print('File "'+str(caller.file)+', Line '+str(caller.rel_lineno))

def step(b):
    a.value+=1

def backstep(b):
    a.value -=1

def break_here(breakpoints, step):
    global log
    for breakpoint in breakpoints:
        if breakpoint.break_here(step,log,None,False):
            return True
    return False

def continue_exec(b):
    global breakpoints
    while a.value<a.max:
        a.value+=1
        if break_here(breakpoints,a.value):
            break
        time.sleep(0.1)

def reverse_exec(b):
    global breakpoints
    while a.value>0:
        a.value-=1
        if a.value in breakpoints:
            break
        time.sleep(0.1)

def parse_breakpoint_input(input,log):
    try:
        inputs =input.split()
        fun_name = inputs[0].replace(" ","")
        abs_lineno = int(inputs[1])
        breaker = Breakpoint("line",abs_lineno,fun_name,None,log[abs_lineno].file)
        return breaker
    except:
        return None

def insert_breakpoint(breakpoint):
    global breakpoints,log,add_breakpoint_input
    try:
        parsed_breakpoint = parse_breakpoint_input(breakpoint.tooltip,log)
        if parsed_breakpoint:
            breakpoints.add(parsed_breakpoint)
            breakpoint.description = 'Added '+breakpoint.tooltip
            add_breakpoint_input.value=''
        else:
            breakpoint.description = 'Could not instantiate breakpoint'
    except:
        breakpoint.description = 'Enter a valid Breakpoint'

    breakpoint.value=''

out = widgets.interactive_output(code_display,{'step':a})
out2 = widgets.interactive_output(var_display,{'step':a})
out3 = widgets.interactive_output(insert_breakpoint,{'breakpoint':add_breakpoint_input})

def remove_breakpoint(breakpoint):
    global breakpoints,remove_breakpoint_input
    try:
        inputs =breakpoint.tooltip.split()
        fun_name = inputs[0]
        abs_lineno = int(inputs[1])
        for breakpt in breakpoints:
            if breakpt.line_number == abs_lineno and breakpt.func_name == fun_name:
                breakpoints.remove(breakpt)
                break

        breakpoint.description='Removed '+str(breakpoint.tooltip)
        remove_breakpoint_input.value = ''
    except:
        breakpoint.description='Enter valid breakpoint'
    breakpoint.value=''


step_btn = widgets.Button(
    description='',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='step-forward'
)

backstep_btn = widgets.Button(
    description='',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='step-backward'
)

contn_btn = widgets.Button(
    description='',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='fast-forward'
)

add_breakpoint_button = widgets.Button(
    description='Add Breakpoint',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='',
    icon='check'
)

remove_breakpoint_button = widgets.Button(
    description='Remove Breakpoint',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='',
    icon='check'
)

callstep_down_button = widgets.Button(
    description='',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Callstack down',
    icon='step-forward'
)

callstack_up_button = widgets.Button(
    description='',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Callstack up',
    icon='step-backward'
)

add_watchpoint_button = widgets.Button(
    description='Add Watchpoint',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Add Watchpoint',
    icon='check'
)

remove_watchpoint_button = widgets.Button(
    description='Remove Watchpoint',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Remove Watchpoint',
    icon='check'
)

add_watchpoint_input = widgets.Text(
    value='',
    placeholder='Enter: <varname> to add a watchpoint',
    description='',
    disabled=False
)

remove_watchpoint_input = widgets.Text(
    value='',
    placeholder='Enter: <varname> to remove a watchpoint',
    description='',
    disabled=False
)

def watchpoint_display(step,layer1,layer2):
    global log
    print('\nWatchpoints: ')
    current_execution  = log[step]
    for watchpoint in watchpoints:
        if watchpoint in current_execution.changed_vars:
            if step>0:
                if watchpoint in log[step-1].changed_vars:
                    if log[step-1].changed_vars[watchpoint] != current_execution.changed_vars[watchpoint]:
                        print(watchpoint+ ': '+str(current_execution.changed_vars[watchpoint]))
                else:
                    print(watchpoint + ': '+str(current_execution.changed_vars[watchpoint]))


callstack_textfield = widgets.HTML(description='', value='')
watchpoints_textfield = widgets.HTML(description='',value='')
out4 = widgets.interactive_output(callstack_display,{'step':a,'layer':callstack_textfield})
out5 = widgets.interactive_output(watchpoint_display,{'step':a,'layer1':add_watchpoint_input,'layer2':remove_watchpoint_input})

def insert_watchpoint(b):
    global watchpoints, add_watchpoint_input
    watchpoints.add(b.tooltip)
    b.description = 'Added watchpoint '+b.tooltip
    add_watchpoint_input.value = ''

def remove_watchpoint(b):
    global watchpoints,add_watchpoint_input
    try:
        watchpoints.remove(b.tooltip)
        b.description = 'Removed watchpoint '+b.tooltip
    except:
        b.description = 'Enter a valid watchpoint'

    remove_watchpoint_input.value = ''


def move_callstack_up(change):
    global current_depth,current_callstack
    if current_depth>0:
        current_depth=current_depth-1
    new_value = ''
    call = current_callstack[current_depth]
    for i in range(0,len(call.fun_code)):
        if call.rel_lineno-1==i:
            new_value = new_value + ("<h4> >> "+call.fun_code[i]+"/h4>\n")
        else:
            new_value = new_value + ("<h4>"+call.fun_code[i]+"/h4>\n")
    callstack_textfield.value = new_value

def move_callstack_down(change):
    global current_depth,current_callstack
    if current_depth<len(current_callstack)-1:
        current_depth=current_depth+1
#    callstack_textfield.value = 'Current Depth: '+str(current_depth)
    new_value = ''
    call = current_callstack[current_depth]
    for i in range(0,len(call.fun_code)):
        if call.rel_lineno-1==i:
            new_value = new_value + ("<h4> >> "+call.fun_code[i]+"/h4>\n")
        else:
            new_value = new_value + ("<h4>"+call.fun_code[i]+"/h4>\n")
    callstack_textfield.value = new_value

def handle_callstack_change(callstack_button):
    callstack_button.tooltip=callstack_button.tooltip+'e'

callstack_up_button.on_click(handle_callstack_change)

callstack_up_button.observe(move_callstack_up,names='tooltip')

callstep_down_button.observe(move_callstack_down,names='tooltip')

callstack_up_button.on_click(handle_callstack_change)
callstep_down_button.on_click(handle_callstack_change)

widgets.link((add_breakpoint_input,'value'),(add_breakpoint_button,'tooltip'))
widgets.link((remove_breakpoint_input,'value'),(remove_breakpoint_button,'tooltip'))

widgets.link((add_watchpoint_input,'value'),(add_watchpoint_button,'tooltip'))
widgets.link((remove_watchpoint_input,'value'),(remove_watchpoint_button,'tooltip'))

add_watchpoint_button.on_click(insert_watchpoint)
remove_watchpoint_button.on_click(remove_watchpoint)

reverse_btn = widgets.Button(
    description='',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='fast-backward'
)

step_btn.on_click(step)
backstep_btn.on_click(backstep)
contn_btn.on_click(continue_exec)
reverse_btn.on_click(reverse_exec)
add_breakpoint_button.on_click(insert_breakpoint)
remove_breakpoint_button.on_click(remove_breakpoint)

widgets.HBox([widgets.VBox([a,contn_btn,reverse_btn,step_btn,backstep_btn,add_breakpoint_input,add_breakpoint_button,remove_breakpoint_input,remove_breakpoint_button,add_watchpoint_input,add_watchpoint_button,remove_watchpoint_input,remove_watchpoint_button]),out, widgets.VBox([out2,out5,out4,callstack_textfield,callstack_up_button,callstep_down_button])])